## Задача №2

### 1. Оценка максимального правдоподобия для $\theta=p_1-p_2$:

По определению оценки максимального правдоподобия: 

$$\widehat\theta = \underset{\theta}{argmax}(\ln(L_{\theta}))=\underset{\theta}{argmax}\left(\ln\left(p_1^{X_1}(1-p_1)^{N_1-X_1}\cdot p_2^{X_2}(1-p_2)^{N_2-X_2}\right)\right) = \underset{\theta}{argmax}\left(X_1\ln(p_1) + (N_1-X_1)\ln(1-p_1) + X_2\ln(p_2) + (N_2-X_2)\ln(1-p_2)\right),$$

где $L_{\theta}$ -- логарифмическая функция потерь.

Найдем $p_1$ и $p_2$, приравняв к нулю производные логарифмической функции правдоподобия по $p_1$ и $p_2$ соответственно:

$$0 = \frac{\partial L_{\theta}}{\partial p_1} = \frac{X_1}{p_1} - \frac{N_1 - X_1}{1 - p_1} \Rightarrow p_1 = \frac{X_1}{N_1}$$
$$0 = \frac{\partial L_{\theta}}{\partial p_2} = \frac{X_1}{p_2} - \frac{N_2 - X_2}{1 - p_2} \Rightarrow p_2 = \frac{X_2}{N_2}$$

Теперь можем найти $\widehat\theta$: $\theta=p_1-p_2$, тогда
$$\widehat{\theta} = \frac{X_1}{N_1} - \frac{X_2}{N_2}$$



### 2. Состоятельная оценка для $\mathrm{D}\widehat{\theta}$:

$$\mathrm{D}\left(\frac{X_1}{N_1}\right) = \frac{p_1(1 - p_1)}{N_1} \xrightarrow{P_{p_1}} \frac{X_1(N_1 - X_1)}{N_1^3},$$
$$\mathrm{D}\left(\frac{X_2}{N_2}\right) = \frac{p_2(1 - p_2)}{N_2} \xrightarrow{P_{p_2}} \frac{X_2(N_2 - X_2)}{N_2^3},$$

поэтому 
$$\mathrm{D}\widehat{\theta} = \mathrm{D}\left(\frac{X_1}{N_1}\right) + \mathrm{D}\left(\frac{X_2}{N_2}\right) 
= \frac{X_1(N_1-X_1)}{N_1^3} + \frac{X_2(N_2-X_2)}{N_2^3}$$

### 3. Асимптотические доверительные интервалы для $𝑝_1$ и $𝑝_2$:

По ЦПТ:
$$\frac{\frac{X_1}{N_1} - p_1}{\sqrt{N_1\frac{X_1}{N_1}(1 - \frac{X_1}{N_1})}}\xrightarrow{d}\frac{\frac{X_1}{N_1} - p_1}{\sqrt{N_1\cdot p_1(1 - p_1)}}\xrightarrow{d}\mathcal{N}(0,1),$$

откуда

$$\mathrm{P}\left(-1 + \frac{\varepsilon}{2} < \frac{\frac{X_1}{N_1} - p_1}{\sqrt{N_1\frac{X_1}{N_1}(1 - \frac{X_1}{N_1})}}<1 - \frac{\varepsilon}{2}\right) = $$

$$\mathrm{P}\left(\frac{X_1}{N_1} - (1 - \frac{\varepsilon}{2})\cdot\sqrt{\frac{\frac{X_1}{N_1}(1 - \frac{X_1}{N_1})}{N_1}}<p_1<\frac{X_1}{N_1}+(1 - \frac{\varepsilon}{2})\cdot\sqrt{\frac{\frac{X_1}{N_1}(1 - \frac{X_1}{N_1})}{N_1}}\right) \rightarrow$$

$$ \rightarrow 2\Phi(1 - \frac{\varepsilon}{2})-1.$$

Для $p_2$ аналогично.

Доверительный интервал ~95% для $\theta$ получается:

$$\left(\widehat{\theta}-2\sqrt{\mathrm{D}\widehat{\theta}}, \ \widehat{\theta}+2\sqrt{\mathrm{D}\widehat{\theta}} \right)$$

### 4. Построение интервалов, используя бутстрепную оценку дисперсии:

In [8]:
import numpy as np

Значения N1, N2, X1, X2:

In [13]:
arguments = [[58, 89, 10, 12], [58, 89, 15, 12], [58, 89, 21, 12]]

в samples[i] запишем данные, соответствующие i-й выборке данных X1, N1, X2, N2;

In [14]:
samples = [[] for i in range(len(arguments))]

for i in range(len(arguments)):
    N1 = arguments[i][0]
    N2 = arguments[i][1]
    X1 = arguments[i][2]
    X2 = arguments[i][3]
    
    samples[i].append(np.hstack([np.zeros(N1 - X1), np.ones(X1)]))
    samples[i].append(np.hstack([np.zeros(N2 - X2), np.ones(X2)]))

Генерация bootstrap'ной выборки:

In [22]:
def get_bootstrap_sample(sample):   
    new_sample_value = np.floor(np.random.rand(len(sample)) * len(sample)).astype(int)
    new_sample = np.array(sample[new_sample_value])
    return new_sample

Функция, по i возвращающая $\widehat{\theta}$ по выборкам sample1, sample2:

In [29]:
def get_thetta(sample1, sample2):
    this_X1 = np.count_nonzero(sample1)
    this_X2 = np.count_nonzero(sample2)
    
    return float(this_X1) / len(sample1) - float(this_X2) / len(sample2)

Вычисление дисперсии выборки samples[i]:

In [32]:
def get_bootstrap_variance(i):
    thettas = []
    for j in range(1000):
        thettas.append(get_thetta(get_bootstrap_sample(samples[i][0]),\
                                  get_bootstrap_sample(samples[i][1])))
        
    return np.var(thettas)

#### Построение выборочной оценки дисперсии оценки параметра $\theta$ для каждого набора данных и сравнение с данными, полученными теоретически:

In [35]:
for i in range(len(arguments)):
    print "Number of data:", i + 1
    
    N1 = arguments[i][0]
    N2 = arguments[i][1]
    X1 = arguments[i][2]
    X2 = arguments[i][3]
    
    print 'N1:', N1, 'X1:', X1, 'N2:', N2, 'X2:', X2
    print 'Theory:', float(X1 * (N1 - X1)) / (N1 ** 3) + float(X2 * (N2 - X2)) / (N2 ** 3)
    print 'Bootstrap:', get_bootstrap_variance(i)
    print

Number of data: 1
N1: 58 X1: 10 N2: 89 X2: 12
Theory: 0.00377082139771
Bootstrap: 0.00378944364538

Number of data: 2
N1: 58 X1: 15 N2: 89 X2: 12
Theory: 0.00461648952678
Bootstrap: 0.00456192270997

Number of data: 3
N1: 58 X1: 21 N2: 89 X2: 12
Theory: 0.00529302403004
Bootstrap: 0.00530314759919



#### Построение доверительного интервала $(\widehat{\theta} - 2\sigma, \widehat{\theta} - 2\sigma)$, используя в качестве $\sigma$ бутстрепную дисперсию. 
P -- вероятность покрытия $\theta$ этим интервалом.

In [39]:
for i in range(len(arguments)):
    print "Number of data:", i + 1
    
    N1 = arguments[i][0]
    N2 = arguments[i][1]
    X1 = arguments[i][2]
    X2 = arguments[i][3]
    
    print 'N1:', N1, 'X1:', X1, 'N2:', N2, 'X2:', X2
    thetta = get_thetta(samples[i][0], samples[i][1])
    variance = get_bootstrap_variance(i)

    good_cases_number = 0
    for j in range(10000):
        bootstrap_thetta = get_thetta(get_bootstrap_sample(samples[i][0]), \
                           get_bootstrap_sample(samples[i][1]))
        
        # if inside confidence segment
        if abs(bootstrap_thetta - thetta) < 2 * np.sqrt(variance):
            good_cases_number += 1
            
    print 'Pr[theta is in confidence segment] =', float(good_cases_number) / 10000.
    print

Number of data: 1
N1: 58 X1: 10 N2: 89 X2: 12
Pr[theta is in confidence segment] = 0.9542

Number of data: 2
N1: 58 X1: 15 N2: 89 X2: 12
Pr[theta is in confidence segment] = 0.9532

Number of data: 3
N1: 58 X1: 21 N2: 89 X2: 12
Pr[theta is in confidence segment] = 0.9629

